# Make figures for the GEOS 505 Climate Dashboard

Front End Team

Fall 2022

In [1]:
import pandas as pd
import numpy as np
import os
import panel as pn
import geopandas as gpd
pn.extension()
import xarray as xr
import rasterio as rio
import folium as fm
import matplotlib
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
import branca
import glob
import io
from PIL import Image

## Define paths in directory, load data

In [2]:
# -----Path to data
data_path = '/Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/'

# -----Path where outputs will be saved
out_path = data_path + '../GEOS505-front-end-team/figures/'

# -----Load shapefile for clipping 
shp_path = data_path + '../doi_12_unified_regions_20180801_shapefile/DOI_12_Unified_Regions_20180801_WGS84.shp'
shp_fn = glob.glob(shp_path)[0]
shp = gpd.read_file(shp_fn)
# shp = shp.to_crs(4326) # reproject to WGS84

# -----Load data files as xarray.Datasets
os.chdir(data_path)
ds_prate = xr.open_dataset('cfs_prate_20221130.nc') # precipitation rate [kg m**-2 s**-1]
ds_sde = xr.open_dataset('cfs_sde_20221130.nc') # snow depth [m]
ds_t = xr.open_dataset('cfs_t_20221130.nc')     # air temperature [K]
ds_wr = xr.open_dataset('cfs_watr_20221130.nc') # water runoff [kg m**-2]

# -----Merge Datasets
ds = xr.merge([ds_prate, ds_sde, ds_t, ds_wr])
# replace no data values with NaN
for data_var in ds.data_vars:
    ds[data_var] = ds[data_var].where(ds[data_var]!=ds[data_var].GRIB_missingValue)
    
# -----Convert units
ds_convert = ds.copy()
rho_w = 1000 # [kg m^-3]
ds_convert['prate'] = ds_convert['prate'] / rho_w * 86400 *1000 # [mm/d]
ds_convert['sde'] = ds_convert['sde'] * 1000 # [mm]
ds_convert['t'] = ds_convert['t'] - 273.15 # [C]
ds_convert['watr'] = ds_convert['watr'] / rho_w * 1000 # [mm]

### Subset the data to a unified region

In [ ]:
# create drop down bar to select region
# region_names = sorted(list(shp['REG_NAME']))
# select_region = pn.widgets.Select(name='Select region:', options=region_names)
# select_region

In [ ]:
# from shapely.geometry import Polygon
# from shapely.geometry import Polygon
# subset the shapefile according to the selector_region value
# shp_subset = shp.loc[shp['REG_NAME']==select_region.value]
# # modify longitude values to match the dataset
# lon, lat = shp_subset.geometry.reset_index(drop=True)[0].exterior.coords.xy
# lon, lat = np.array(lon, dtype=float), np.array(lat, dtype=float)
# lon[lon<0] = lon[lon<0] + 360
# # reformat as Polygon
# poly = Polygon(list(zip(lon, lat)))
# 
# # mask the data using the region shape
# mask = rio.features.geometry_mask([poly],
#                                   out_shape=(len(ds['latitude'].data), len(ds['longitude'].data)),
#                                   transform=shp.transform,
#                                   invert=False)
# mask = xr.DataArray(mask, dims=("latitude", "longitude"))
# # mask DEM values outside the AOI
# ds_subset = ds_subset.where(~mask)
# ds_subset

In [ ]:
# fig, ax = plt.subplots()
# ds['t'].isel(valid_time=0).plot(ax=ax)
# ax.plot(*poly.exterior.coords.xy, 'k')
# ax.set_xlim(200, 300)
# ax.set_ylim(0, 90)
# plt.show()

In [3]:
# grab a spatial subset of the PNW
ds_convert_PNW = ds_convert.where((ds_convert.latitude > 40) & (ds_convert.latitude < 50)
                  & (ds_convert.longitude > 230) & (ds_convert.longitude < 255), drop=True)
ds_convert_PNW

<xarray.Dataset>
Dimensions:     (valid_time: 28, latitude: 11, longitude: 27)
Coordinates:
    time        datetime64[ns] 2022-11-30
    step        (valid_time) timedelta64[ns] 0 days 00:00:00 ... 6 days 18:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 49.61 48.66 47.72 46.77 ... 42.05 41.1 40.16
  * longitude   (longitude) float64 230.6 231.6 232.5 ... 253.1 254.1 255.0
  * valid_time  (valid_time) datetime64[ns] 2022-11-30 ... 2022-12-06T18:00:00
Data variables:
    prate       (valid_time, latitude, longitude) float64 9.418 12.53 ... 17.19
    sde         (valid_time, latitude, longitude) float32 nan nan ... 56.26
    t           (valid_time, latitude, longitude) float32 10.59 9.979 ... -0.639
    watr        (valid_time, latitude, longitude) float32 nan nan ... 0.00608

### Create interactive map and scatterplot for each of the data variables

In [4]:
# -----Create drop down bar to select forecast timeframe
select_timeframe = pn.widgets.Select(name='Select forecast timeframe:', options=['Day', 'Week', 'Month'])
select_timeframe

Select(options=['Day', 'Week', 'Month'], value='Day')

In [5]:
# -----Set up for plotting
### Identify start and end times
# for the sample dataset, we'll set today as the first valid_time in the dataset
# t_start = np.datetime64('today')
t_start = ds_convert_PNW['valid_time'].data[0]
if select_timeframe.value=='Day':
    t_end = t_start + np.timedelta64(1, 'D')
elif select_timeframe.value=='Week':
    t_end = t_start + np.timedelta64(7, 'D')
elif select_timeframe.value=='Month':
    t_end = t_start + np.timedelta64(30, 'D')
### Calculate mean of each variable over time (for map)
ds_convert_PNW_time_mean = ds_convert_PNW.sel(valid_time=slice(t_start, t_end)).mean(dim='valid_time')
### Calculate mean and standard deviation of each variable over space (for line chart)
ds_convert_PNW_space_mean = ds_convert_PNW.sel(valid_time=slice(t_start, t_end)).mean(dim=['latitude', 'longitude'])
ds_convert_PNW_space_std = ds_convert_PNW.sel(valid_time=slice(t_start, t_end)).std(dim=['latitude', 'longitude'])
### Plot settings
# data variables to plot
data_vars = ['prate', 'sde', 't', 'watr']
# display names for each data variable
data_vars_display = ['Preciptiation rate [mm/d]', 
                     'Snow depth [mm]', 
                     'Air temperature [F]', 
                     'Water runoff [mm]']
# colormaps for map
cmaps = [matplotlib.cm.get_cmap('Blues'), 
         matplotlib.cm.get_cmap('cool'),
         matplotlib.cm.get_cmap('coolwarm'),
         matplotlib.cm.get_cmap('GnBu')]
# colors for line charts
chart_colors = [cmaps[0](150),
                cmaps[1](50),
                cmaps[2](150),
                cmaps[3](150)
               ]
# min and max of map
xmin, xmax = np.min(ds_convert_PNW.longitude.data), np.max(ds_convert_PNW.longitude.data)
ymin, ymax = np.min(ds_convert_PNW.latitude.data), np.max(ds_convert_PNW.latitude.data)
# function to "colorize" the data
# from: https://www.linkedin.com/pulse/visualize-dem-interactive-map-chonghua-yin/?trk=related_artice_Visualize%20DEM%20in%20An%20Interactive%20Map_article-card_title
def colorize(array, cmap):
    normed_data = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array))   
    cm = cmap
    return cm(normed_data)

# -----Loop through data variables
for i, data_var in enumerate(data_vars):
    
    print(data_var)
    print('---------')
    
    # -----Create line chart
    # create pandas.DataFrame
    df = pd.DataFrame()
    df['valid_time'] = ds_convert_PNW_space_mean.valid_time.data
    df['mean'] = ds_convert_PNW_space_mean[data_var].data
    df['mean-std'] = ds_convert_PNW_space_mean[data_var].data - ds_convert_PNW_space_std[data_var].data
    df['mean+std'] = ds_convert_PNW_space_mean[data_var].data + ds_convert_PNW_space_std[data_var].data
    # line plot for median, min, and max, area plot for std
    std_plot = df.hvplot.area(x='valid_time', y='mean-std', y2='mean+std', label='standard deviation', fill_color=chart_colors[i], line_color=chart_colors[i])
    med_plot = df.hvplot.line(x='valid_time', y='mean', label='mean', color='black')
    list_of_curves = [std_plot, med_plot]
    # add list of curves to plot
    chart = hv.Overlay(list_of_curves).opts(
        height=500, 
        width=800,
        ylabel=data_vars_display[i],
        xlabel='',
        title='Mean and standard deviation of ' + data_var + ' for the next ' + str(select_timeframe.value),
        legend_position='bottom_right',
    )
    # save chart to file
    fn = out_path + 'chart_'+data_var+'.html'
    hvplot.save(chart, fn)
    print('chart saved to file: '+fn)
    
    # -----Create map
    m = fm.Map(location=[np.nanmean(ds_convert_PNW.latitude.data),  # mean latitude value in data 
                         np.nanmean(ds_convert_PNW.longitude.data)], # mean longitude value in data
                      zoom_start=4, # initial map zoom level
                      tiles='StamenTerrain', # basemap
                      width=500, # map width
                      height=400) # map height
    # create colormap for legend
    cmap_legend = branca.colormap.LinearColormap([cmaps[i](j) for j in np.arange(0,256)], 
                                                 vmin=np.nanmin(ds_convert_PNW_time_mean[data_var]), 
                                                 vmax=np.nanmax(ds_convert_PNW_time_mean[data_var]), 
                                                 caption=data_vars_display[i], 
                                                 tick_labels=[np.nanmin(ds_convert_PNW_time_mean[data_var].data),
                                                              np.nanmax(ds_convert_PNW_time_mean[data_var].data),
                                                              np.nanmax(ds_convert_PNW_time_mean[data_var].data)]
                                                )
    # colorize the data
    data_colorized = colorize(ds_convert_PNW_time_mean[data_var].data, cmaps[i])
    # add image to map
    fm.raster_layers.ImageOverlay(image=data_colorized, 
                                  bounds=[[ymin, xmin], [ymax, xmax]], 
                                  opacity=0.8,
                                  origin='upper', 
                                ).add_to(m)
    # add colormap legend to map
    m.add_child(cmap_legend)
    # save to file as png
    fn = out_path + 'map_' + data_var + ".html"
    m.save(fn)
    print('map saved to file: ' + fn)
    print(' ')


prate
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/../GEOS505-front-end-team/figures/chart_prate.html
map saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/../GEOS505-front-end-team/figures/map_prate.html
 
sde
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/../GEOS505-front-end-team/figures/chart_sde.html
map saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/../GEOS505-front-end-team/figures/map_sde.html
 
t
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/../GEOS505-front-end-team/figures/chart_t.html
map saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/../GEOS505-front-end-team/figures/map_t.html
 
watr
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dashboard/data/../GEOS505-front-e

__NOTE:__ The Folium maps and charts can be added to the Panel dashboard as in the following command (like [this example](https://panel.holoviz.org/gallery/external/Folium.html)):

`pn.panel(map, height=400)`